In [1]:
import numpy as np
import pandas as pd

In [2]:
# 1. 데이터 불러오기

In [3]:
train = pd.read_csv('/content/drive/MyDrive/DayCon_Practice/따릉이/train.csv')
test = pd.read_csv('/content/drive/MyDrive/DayCon_Practice/따릉이/test.csv')

In [4]:
# 2. 데이터 확인

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


In [7]:
# 3. 결측치 처리

In [8]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [9]:
train.fillna(train.mean(),inplace=True)
train.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [10]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [11]:
test.fillna(test.mean(),inplace=True)
test.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
dtype: int64

In [12]:
# 4. 입력 변수 종속 변수 분리

In [13]:
X_train = train.drop('count',axis=1)
y_train = train['count']
X_test = test

In [14]:
# 5. 모델 불러오기

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

In [16]:
# train_validation_split

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=10)

In [18]:
models = {'DT': DecisionTreeRegressor(),
          'RF': RandomForestRegressor(),
          'LGB': lgb.LGBMRegressor(),
          'XGB': xgb.XGBRegressor(),
          'KNN': KNeighborsRegressor()}

In [19]:
from sklearn.metrics import mean_squared_error

for model_name in models.keys():
  model = models[model_name]
  model.fit(X_train,y_train)
  y_pred = model.predict(X_val)

  RMSE = mean_squared_error(y_val, y_pred)**0.5
  print(f'{model_name} RMSE: {RMSE}')

DT RMSE: 46.803472825135
RF RMSE: 41.70327593403349
LGB RMSE: 40.951888829466725
[13:12:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB RMSE: 43.49030918124216
KNN RMSE: 80.05216278854238


In [20]:
# 다시 훈련 데이터 전체 사용
X_train = train.drop('count',axis=1)
y_train = train['count']
X_test = test

In [21]:
#lgb - no tunning 제출
model = lgb.LGBMRegressor()
model.fit(X_train,y_train)
y_test = model.predict(X_test)

submission = pd.read_csv('/content/drive/MyDrive/DayCon_Practice/따릉이/submission.csv')
submission['count'] = y_test
submission.to_csv('lgb.csv',index=False)

In [22]:
#lgb - gridsearchCV tunning 
from sklearn.model_selection import GridSearchCV

model = lgb.LGBMRegressor()
parameters = {'num_leaves':[20,40,60,80,100], 'min_child_samples':[5,10,15],'max_depth':[-1,5,10,20],
             'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}

reg = GridSearchCV(model,parameters,scoring='neg_mean_squared_error')
reg.fit(X_train, y_train)
print('final params', reg.best_params_)   # 최적의 파라미터 값 출력
print('best score', reg.best_score_)

final params {'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 5, 'num_leaves': 40, 'reg_alpha': 0.01}
best score -1401.1138898865522


In [23]:
#lgb - tunning 제출
model = lgb.LGBMRegressor(learning_rate=0.1,max_depth=-1,min_child_samples=5,num_leaves=40,reg_alpha=0.01)
model.fit(X_train,y_train)
y_test = model.predict(X_test)

submission = pd.read_csv('/content/drive/MyDrive/DayCon_Practice/따릉이/submission.csv')
submission['count'] = y_test
submission.to_csv('lgb_with_tunning.csv',index=False)

## 0.8 LightGB + 0.2 XGB

In [25]:
# LightGB
model = lgb.LGBMRegressor()
model.fit(X_train,y_train)
y_test_LGB = model.predict(X_test)

In [26]:
# XGB
model = xgb.XGBRegressor()
model.fit(X_train,y_train)
y_test_XGB = model.predict(X_test)

[13:30:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
# 0.8 LightGB + 0.2 XGB
y_test = y_test_LGB * 0.8 + y_test_XGB * 0.2
submission = pd.read_csv('/content/drive/MyDrive/DayCon_Practice/따릉이/submission.csv')
submission['count'] = y_test
submission.to_csv('0.8lgb_0.2xgb.csv',index=False)